## Image to Image Inference using Online Endpoints

This sample shows how to deploy `image to image` type stable diffusion models to a batch endpoint for inference.

### Task
`image to image` task takes an original image and a text prompt as input. The model generates an image by modifying the original image.

 
### Model
Models that can perform the `image to image` task are tagged with `image-to-image`. We will use the `stabilityai-stable-diffusion-xl-refiner-1-0` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name.


### Outline
1. Setup pre-requisites
2. Pick a model to deploy
3. Prepare data for inference - using a folder of csv files with prompt, image columns
4. Deploy the model to a batch endpoint
5. Test the endpoint - using csv files
6. Clean up resources - delete the endpoint

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<AML_WORKSPACE_NAME>"

workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
registry_name = "azureml"
registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name=registry_name,
)

#### Create a compute cluster
Use the model card from the AzureML system registry to check the minimum required inferencing SKU, referenced as size below. If you already have a sufficient compute cluster that you wish to use, you can simply define the name in `compute_name` in the following code block. Otherwise, the below snippet will create a new compute cluster.

In [ ]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "gpu-cluster"

try:
    _ = workspace_ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        description="An AML compute cluster",
        size="STANDARD_NC4AS_T4_V3",
        min_instances=0,
        max_instances=3,
        idle_time_before_scale_down=120,
    )
    workspace_ml_client.begin_create_or_update(compute_config).result()

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `image-to-image` task. In this example, we use the `stabilityai-stable-diffusion-xl-refiner-1-0` model. If you have opened this notebook for a different model, replace the model name accordingly. This is a pre-trained model.

In [ ]:
# Name of the image to image model to be deployed
model_name = "stabilityai-stable-diffusion-xl-refiner-1-0"

try:
    model = registry_ml_client.models.get(name=model_name, label="latest")
except Exception as ex:
    print(
        f"No model named {model_name} found in registry. "
        "Please check model name present in Azure model catalog"
    )
    raise ex

print(
    f"\n\nUsing model name: {model.name}, version: {model.version}, id: {model.id} for generating images from text."
)

### 3. Prepare data for inference - using a folder of csv files with prompt and image columns

The CSV files should consist of 2 columns namely, 
* `image`: Original image to be used as base image for image to image generation. It should either be in base64 format or publicly accessible URL.
* `prompt`: A text prompt.

We provide the text prompts in a csv file starting from the first row of a column named "prompt". We provide the input image as base64 string starting from the first row of the column "image".

The deployment in the "Create batch deployment" section below takes the argument `mini_batch_size`, which is the number of CSV files processed by the model in a single mini_batch. To limit the number of prompts processed in each mini_batch we split the dataset into multiple csv files.

In [ ]:
# Prepare input csv file

import base64
import os
import pandas as pd


def read_image(image_path: str) -> bytes:
    """Reads an image from a file path into a byte array."""
    with open(image_path, "rb") as f:
        return f.read()


base_image1 = "inpainting_data/images/dog_on_bench.png"
base_image2 = "inpainting_data/images/teapot.png"

dataset_parent_dir = "inpainting_data/batch_data"
os.makedirs(dataset_parent_dir, exist_ok=True)

input_data = {
    "columns": ["image", "prompt"],
    "data": [
        {
            "image": base64.encodebytes(read_image(base_image1)).decode("utf-8"),
            "prompt": "A yellow cat, high resolution, sitting on a park bench",
        },
        {
            "image": base64.encodebytes(read_image(base_image2)).decode("utf-8"),
            "prompt": "A small flower featuring a blend of pink and purple colors.",
        },
    ],
}
pd.DataFrame(**input_data).to_csv(
    os.path.join(dataset_parent_dir, "input1.csv"), index=False
)

input_data = {
    "columns": ["image", "prompt"],
    "data": [
        {
            "image": base64.encodebytes(read_image(base_image1)).decode("utf-8"),
            "prompt": "Pikachu, cinematic, digital art, sitting on bench",
        },
        {
            "image": base64.encodebytes(read_image(base_image2)).decode("utf-8"),
            "prompt": "A woman with red hair in the style of Tamara de Lempicka.",
        },
    ],
}
pd.DataFrame(**input_data).to_csv(
    os.path.join(dataset_parent_dir, "input2.csv"), index=False
)

In [ ]:
# Read all the csvs in the data folder into a pandas dataframe
import glob
import os
import pandas as pd

# Specify the folder where your CSV files are located
dataset_parent_dir = "inpainting_data/batch_data"

# Use glob to get a list of CSV files in the folder
csv_files = glob.glob(os.path.join(dataset_parent_dir, "*.csv"))

# Read all CSV files into a single DataFrame using pd.concat
batch_df = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Now, 'batch_df' contains all the data from the CSV files in the folder
print(batch_df.head())

In [ ]:
from pathlib import Path

# Specify the folder where your CSV files should be saved
processed_dataset_parent_dir = "inpainting_data/processed_batch_data"
os.makedirs(processed_dataset_parent_dir, exist_ok=True)
batch_input_file = "batch_input.csv"

# Divide this into files of <x> rows each
batch_size_per_predict = 2
for i in range(0, len(batch_df), batch_size_per_predict):
    j = i + batch_size_per_predict
    batch_df[i:j].to_csv(
        os.path.join(processed_dataset_parent_dir, str(i) + batch_input_file)
    )

# Check out the first and last file name created
input_paths = sorted(Path(processed_dataset_parent_dir).iterdir(), key=os.path.getmtime)
input_files = [os.path.basename(path) for path in input_paths]
print(f"{input_files[0]} to {str(i)}{batch_input_file}.")

Register folder containing csv files in AML as data asset to use in batch job.

In [ ]:
input = Input(path=processed_dataset_parent_dir, type=AssetTypes.URI_FOLDER)

### 4. Deploy the model to a batch endpoint
Batch endpoints are endpoints that are used to do batch inferencing on large volumes of data over a period of time. The endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis. For more information on batch endpoints and deployments, see <a href="https://learn.microsoft.com/en-us/azure/machine-learning/concept-endpoints?view=azureml-api-2#what-are-batch-endpoints" target="_blank"> What are batch endpoints?</a> In this sub-section, we will cover the following items:

* Create a batch endpoint.
* Create a batch deployment.
* Set the deployment as default. Doing so allows invoking the endpoint without specifying the deployment's name.

#### Create a batch endpoint

In [ ]:
import time, sys, uuid
from azure.ai.ml.entities import (
    BatchEndpoint,
    BatchDeployment,
    BatchRetrySettings,
    AmlCompute,
)

# Endpoint names need to be unique in a region,
# hence using uuid (first 8 character) to create unique endpoint name
endpoint_name = (
    "image-to-image-" + str(uuid.uuid4())[:8]
)  # Replace with your endpoint name

# Create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="Batch endpoint for " + model.name + ", for image to image task",
)
workspace_ml_client.begin_create_or_update(endpoint).result()

#### Create a batch deployment

__Note__: `mini_batch_size` is the number of CSV files processed by the model in a single mini_batch.

In [ ]:
deployment_name = "image-to-image-demo"

deployment = BatchDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model.id,
    compute=compute_name,
    error_threshold=0,
    instance_count=1,
    logging_level="info",
    max_concurrency_per_instance=1,
    mini_batch_size=1,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=2, timeout=9999),
)
workspace_ml_client.begin_create_or_update(deployment).result()

#### Set the deployment as default

In [ ]:
endpoint = workspace_ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment_name
workspace_ml_client.begin_create_or_update(endpoint).result()

endpoint = workspace_ml_client.batch_endpoints.get(endpoint_name)
print(f"The default deployment is {endpoint.defaults.deployment_name}")

### 5. Test the endpoint - using csv files

Invoke the batch endpoint with the input parameter pointing to the directory containing one or more csv files containing the batch inference input. This creates a pipeline job using the default deployment in the endpoint. Wait for the job to complete.

__Note__: If job failed with Out of Memory Error then please try splitting your input into smaller csv files or decreasing mini_batch_size for the deployment.

In [ ]:
job = None
num_retries = 3
for i in range(num_retries):
    try:
        job = workspace_ml_client.batch_endpoints.invoke(
            endpoint_name=endpoint.name, input=input
        )
        break
    except Exception as e:
        if i == num_retries - 1:
            raise e
        else:
            print("Endpoint invocation failed. Retrying after 5 seconds...")
            time.sleep(5)
if job is not None:
    workspace_ml_client.jobs.stream(job.name)

__Note__: If the job failed with error Assertion Error (The actual length exceeded max length 100 MB) then please consider dividing input csv file into multiple csv files.

In [ ]:
import pandas as pd

scoring_job = list(workspace_ml_client.jobs.list(parent_job_name=job.name))[0]

workspace_ml_client.jobs.download(
    name=scoring_job.name,
    download_path=".",
    output_name="score",
)

predictions_file = os.path.join("named-outputs", "score", "predictions.csv")

# Load the batch predictions file with no headers into a dataframe and set your column names
score_df = pd.read_csv(
    predictions_file,
    header=None,
    names=[
        "row_number_per_file",
        "image_file_name",
        "nsfw_content_detected",
        "input_csv_name",
    ],
)
score_df

### 6. Clean up resources - delete the endpoint
Batch endpoints use compute resources only when jobs are submitted. You can keep the batch endpoint for your reference without worrying about compute bills, or choose to delete the endpoint. If you created your compute cluster to have zero minimum instances and scale down soon after being idle, you won't be charged for an unused compute.

In [ ]:
workspace_ml_client.batch_endpoints.begin_delete(name=endpoint_name).result()